In [ ]:
!pip install datasets openai google-generativeai google-ai-generativelanguage google-cloud-aiplatform  --upgrade google-api-python-client --upgrade google-cloud

In [3]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

# Load the dataset with the "question-answer" configuration
dataset1 = load_dataset('rag-datasets/rag-mini-wikipedia', 'question-answer')

df1 = pd.DataFrame(dataset1['test'])
# Display the dataset
print(dataset1)

DatasetDict({
    test: Dataset({
        features: ['question', 'answer', 'id'],
        num_rows: 918
    })
})


In [4]:
df1

,question,answer,id
0,Was Abraham Lincoln the sixteenth President of...,yes,0
1,Did Lincoln sign the National Banking Act of 1...,yes,2
2,Did his mother die of pneumonia?,no,4
3,How many long was Lincoln's formal education?,18 months,6
4,When did Lincoln begin his political career?,1832,8
...,...,...,...
913,Was Wilson president of the American Political...,Yes,1710
914,Did he not cast his ballot for John M. Palmer ...,Yes,1711
915,Did Wilson not spend 1914 through the beginnin...,Yes,1712
916,"Was Wilson , a staunch opponent of antisemitis...",Yes,1713


In [5]:
question = df1['question'][917]
print("Question is ",question)
answer  = df1['answer'][917]
print("answer is",answer)

Question is  What happened in 1917?
answer is raised billions through Liberty loans, imposed an income tax, S08_set up the War Industries Board, promoted labor union growth, supervised agriculture and food production through the Lever Act, took over control of the railroads, and suppressed anti-war movements


In [13]:
question1 = df1['question'][3]
print("Question is ",question1)
answer1  = df1['answer'][3]
print("answer is",answer1)

Question is  How many long was Lincoln's formal education?
answer is 18 months


In [7]:
import os
import google.generativeai as genai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

def gemini_response(question):

    # Set your API key
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

    prompt = f'''I want you to answer questions in a concise and informative way, 
            avoiding aggressive words and emoji symbols. 
    "{question}"'''

    # Define the generation configuration
    generation_config = {
        "temperature": 1.0,
        "top_p": 0.95,
        "response_mime_type": "text/plain",
        "top_k": 64,
        "max_output_tokens": 8192
    }

    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    response = model.generate_content(prompt)
    # print(response.text)
    return response.text

In [10]:
result = gemini_response(question)
print(result)

1917 saw the United States enter World War I and the Russian Revolution, leading to the fall of the Tsarist regime.



In [11]:
from rouge_score import rouge_scorer

def calculate_rouge(reference, candidate):

    # Initialize a ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    
    # Extract and format the scores
    rouge_scores = {
        'ROUGE-1': scores['rouge1'].fmeasure,
        'ROUGE-2': scores['rouge2'].fmeasure,
        'ROUGE-L': scores['rougeL'].fmeasure
    }
    
    return rouge_scores

print(calculate_rouge(reference = answer,candidate = result))

{'ROUGE-1': 0.1967213114754098, 'ROUGE-2': 0.033898305084745756, 'ROUGE-L': 0.1967213114754098}


Calculate rougue score for next question


In [15]:
result1 = gemini_response(question1)
print("Result is ",result1)
print(calculate_rouge(reference = answer1,candidate = result1))

Result is  One year.

{'ROUGE-1': 0.0, 'ROUGE-2': 0.0, 'ROUGE-L': 0.0}


In [16]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(reference, candidate):
    
    # Tokenize the reference and candidate texts
    reference_tokens = [reference.split()]
    candidate_tokens = candidate.split()
    
    # Apply smoothing to handle edge cases
    smoothing_function = SmoothingFunction().method1
    
    # Compute BLEU score
    bleu_score = sentence_bleu(
        reference_tokens,
        candidate_tokens,
        smoothing_function=smoothing_function
    )
    
    return bleu_score
bleu = calculate_bleu(reference = answer, candidate=result)
print("BLEU Score:", bleu)

BLEU Score: 0.011314809333648207


In [17]:
bleu = calculate_bleu(reference = answer1, candidate=result1)
print("BLEU Score:", bleu)

BLEU Score: 0
